In [ ]:
import pandas as pd
import numpy as np
test=pd.read_excel("Final_Test_doc.xlsx")
train=pd.read_excel("Final_Train_doc.xlsx")

In [ ]:
#df = pd.concat([x_test, x_train])

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
df_qual_prof = pd.crosstab(train.Qualification, train.Profile, margins=True)
#df_qual_prof.to_csv("df_qual_prof.csv", sep=',')
df_qual_prof.head()

In [ ]:
df_qual_prof.sort_values(by=['All'], ascending=False)

In [ ]:
print('Qualification:', train['Qualification'].nunique())
print('Experience:', train['Experience'].nunique())
print('Rating:', train['Rating'].nunique())
print('Place:', train['Place'].nunique())
print('Profile', train['Profile'].nunique())

In [ ]:
df_train = train[['Qualification', 'Profile', 'Experience', 'Place', 'Miscellaneous_Info', 'Rating','Fees']]
df_test = test[['Qualification', 'Profile','Experience', 'Place', 'Miscellaneous_Info', 'Rating']]

In [ ]:
df_train.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import re
warnings.filterwarnings('ignore')
df_train['Experience'] = df_train['Experience'].str.replace('years experience','').astype(int)
df_test['Experience'] = df_test['Experience'].str.replace('years experience','').astype(int)

In [ ]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

df_train['Qualification'] = df_train['Qualification'].apply(clean_text)
df_test['Qualification'] = df_test['Qualification'].apply(clean_text)

def clean_place(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

df_train['Place'].fillna('missing', inplace=True)
df_test['Place'].fillna('missing', inplace=True)
df_train['Place'] = df_train['Place'].apply(clean_place)
df_test['Place'] = df_test['Place'].apply(clean_place)

In [ ]:
df_train['Rating'].fillna('0%', inplace=True)
df_test['Rating'].fillna('0%', inplace=True)

def clean_rating(text):
    text = re.sub(r'%', '', str(text))
    return text

df_train['Rating'] = df_train['Rating'].apply(clean_rating)
df_train['Rating'] = df_train['Rating'].astype(int)

df_test['Rating'] = df_test['Rating'].apply(clean_rating)
df_test['Rating'] = df_test['Rating'].astype(int)

In [ ]:
df_train['Miscellaneous_Info'].fillna('missing', inplace=True)
df_test['Miscellaneous_Info'].fillna('missing', inplace=True)

def get_feedback(feedback):
    feedback = re.findall(r'\d+ Feedback', str(feedback))
    if feedback == []:
        feedback = '0 Feedback'
        return feedback
    else:
        return feedback[0]

df_train['Feedback'] = df_train['Miscellaneous_Info'].apply(get_feedback)
df_train['Feedback'] = df_train['Feedback'].str.replace(' Feedback','')
df_train['Feedback'] = df_train['Feedback'].astype(int)

df_test['Feedback'] = df_test['Miscellaneous_Info'].apply(get_feedback)
df_test['Feedback'] = df_test['Feedback'].str.replace(' Feedback','')
df_test['Feedback'] = df_test['Feedback'].astype(int)

In [ ]:
def get_fee(text):
    text = re.sub(r',', "", text)
    text = re.findall(r'₹\d+', text)
    if text != []:
        return text[0]
    else:
        return 0

df_train['Misc_Fees'] = df_train['Miscellaneous_Info'].apply(get_fee)
df_train['Misc_Fees'] = df_train['Misc_Fees'].str.replace('₹','')
df_train['Misc_Fees'].fillna(50, inplace=True)
df_train['Misc_Fees'] = df_train['Misc_Fees'].astype(int)

df_test['Misc_Fees'] = df_test['Miscellaneous_Info'].apply(get_fee)
df_test['Misc_Fees'] = df_test['Misc_Fees'].str.replace('₹','')
df_test['Misc_Fees'].fillna(50, inplace=True)
df_test['Misc_Fees'] = df_test['Misc_Fees'].astype(float)

In [ ]:
def clean_misc(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

df_train['Miscellaneous_Info'] = df_train['Miscellaneous_Info'].apply(clean_misc)
df_test['Miscellaneous_Info'] = df_test['Miscellaneous_Info'].apply(clean_misc)

In [ ]:
df_train = pd.get_dummies(df_train, columns=['Profile'])
df_test  = pd.get_dummies(df_test,  columns=['Profile'])

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in df_train.columns:
    if df_train[column_name].dtype == object:
        df_train[column_name] = le.fit_transform(df_train[column_name])

In [ ]:
from sklearn.preprocessing import StandardScaler

s_sc = StandardScaler()
col_to_scale = df_train.columns
df_train[col_to_scale] = s_sc.fit_transform(df_train[col_to_scale])

In [ ]:
from sklearn import preprocessing
def convert(df_train):
    number = preprocessing.LabelEncoder()
    df_train['Miscellaneous_Info'] = number.fit_transform(df_train.Miscellaneous_Info)
    df_train['Qualification'] = number.fit_transform(df_train.Qualification)   
    df_train['Place'] = number.fit_transform(df_train.Place)
    df_train=df_train.fillna(-999)
    return df_train


In [ ]:
df_train

In [ ]:
df_train.skew

In [ ]:
df_train.fillna(df_train.median(), inplace=True)

In [ ]:
X = df_train.drop(labels=['Fees'], axis=1)
y = df_train['Fees'].values

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=1)

In [ ]:
X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

In [ ]:
X_train.columns

In [ ]:
from math import sqrt 
from sklearn.metrics import mean_squared_log_error, mean_squared_error

In [ ]:
#calculating r2 score
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score,roc_curve
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,Y_test=train_test_split(X, y,random_state=20,test_size=0.33)
model=[DecisionTreeRegressor(),KNeighborsRegressor(),AdaBoostRegressor(),LinearRegression(),GradientBoostingRegressor()]
from sklearn.decomposition import PCA
max_r_score=0
for r_state in range(42,100):
    x_train,x_test,y_train,Y_test=train_test_split(X, y,random_state=r_state,test_size=0.33)
    for i in model:
        i.fit(x_train,y_train)
        pre=i.predict(x_test)
        r2_sc=r2_score(Y_test,pre)
        print("max r2 score corrospond to random state",r_state,"is",r2_sc)
        if r2_sc>max_r_score:
            max_r_score=r2_sc
            final_r_state=r_state
            final_model=i
            print()
            print()
            print()
            print()
            print("max r2 score corrospond to random state",final_r_state,"is",max_r_score,"and model is",final_model)
 

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(linear_model.LinearRegression(),df_x,y,cv=10,scoring="r2").mean()

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
print("r2 score is",r2_score(Y_test,y_pred))
print("RMSE is",np.sqrt(mean_squared_error(Y_test,y_pred)))

In [ ]:
import joblib
joblib.dump(lreg,'fees.pkl')
